In [1]:
# 1. Mount Google Drive
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [2]:
# install optuna

!pip install optuna

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 386.6/386.6 kB 34.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 242.5/242.5 kB 26.3 MB/s eta 0:00:00


In [3]:
# 2. Install Ultralytics YOLO if not installed
!pip install ultralytics

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 56.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 70.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 36.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 51.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 12.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 4.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 80.8 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstalling 

In [4]:
# 3. Import required modules
import zipfile
import os
from ultralytics import YOLO
import optuna

Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.


In [5]:
# 4. Define paths
train_zip = '/content/drive/MyDrive/computer_vision/segmentation_task_28_05_2025/dataset/train.zip'
val_zip = '/content/drive/MyDrive/computer_vision/segmentation_task_28_05_2025/dataset/val.zip'
unzip_dir = '/content/drive/MyDrive/computer_vision/segmentation_task_28_05_2025/dataset'

In [6]:
# 5. Unzip train and val if not already unzipped
def unzip_file(zip_path, extract_to):
    with zipfile.ZipFile(zip_path, 'r') as zip_ref:
        zip_ref.extractall(extract_to)

if not os.path.exists(os.path.join(unzip_dir, 'train')):
    unzip_file(train_zip, unzip_dir)

if not os.path.exists(os.path.join(unzip_dir, 'val')):
    unzip_file(val_zip, unzip_dir)

In [7]:
# 6. Path to your config.yaml
data_yaml = '/content/drive/MyDrive/computer_vision/segmentation_task_28_05_2025/dataset/config.yaml'

In [10]:
def objective(trial):
    from ultralytics import YOLO

    # Suggest valid hyperparameters
    lr0 = trial.suggest_float("lr0", 1e-5, 1e-2, log=True)  # correct: initial learning rate
    batch = trial.suggest_categorical("batch", [8, 16, 32])
    weight_decay = trial.suggest_float("weight_decay", 1e-5, 1e-2, log=True)
    imgsz = trial.suggest_categorical("imgsz", [320, 416, 512, 640])
    dropout = trial.suggest_float("dropout", 0.0, 0.5)
    mosaic = trial.suggest_float("mosaic", 0.0, 1.0)
    mixup = trial.suggest_float("mixup", 0.0, 1.0)

    # Load model
    model = YOLO("yolov8n-seg.pt")

    # Train model
    results = model.train(
        data=data_yaml,
        epochs=100,
        imgsz=imgsz,
        batch=batch,
        lr0=lr0,
        weight_decay=weight_decay,
        dropout=dropout,
        mosaic=mosaic,
        mixup=mixup,
        optimizer="Adam",
        project="/content/drive/MyDrive/computer_vision/segmentation_task_28_05_2025/runs",
        name=f"trial_{trial.number}",
        exist_ok=True,
        verbose=False
    )

    # Validate model and return the metric to optimize
    metrics = model.val()
    return metrics.seg.map50  # Return mAP@0.5 for segmentation


In [ ]:
# Run the Optuna study
import optuna
study = optuna.create_study(direction="maximize")
study.optimize(objective, n_trials=20)

print("Best hyperparameters found:")
print(study.best_trial.params)



[I 2025-05-31 20:12:21,828] A new study created in memory with name: no-name-17c0be7e-1c54-4d1c-87a4-e72643faa7d0


Ultralytics 8.3.146 🚀 Python-3.11.12 torch-2.6.0+cu124 CUDA:0 (Tesla T4, 15095MiB)
engine/trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=16, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=/content/drive/MyDrive/computer_vision/segmentation_task_28_05_2025/dataset/config.yaml, degrees=0.0, deterministic=True, device=None, dfl=1.5, dnn=False, dropout=0.3583854828514219, dynamic=False, embed=None, epochs=100, erasing=0.4, exist_ok=True, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=640, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.0008438170308626528, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.07285516457456254, mode=train, model=yolov8n-seg.pt, momentum=0.937, mosaic=0.8884397073075971, multi_scale=False, name=trial_0, nbs=64, nms=False,

100%|██████████| 755k/755k [00:00<00:00, 79.1MB/s]

Overriding model.yaml nc=80 with nc=1

                   from  n    params  module                                       arguments                     
  0                  -1  1       464  ultralytics.nn.modules.conv.Conv             [3, 16, 3, 2]                 
  1                  -1  1      4672  ultralytics.nn.modules.conv.Conv             [16, 32, 3, 2]                
  2                  -1  1      7360  ultralytics.nn.modules.block.C2f             [32, 32, 1, True]             
  3                  -1  1     18560  ultralytics.nn.modules.conv.Conv             [32, 64, 3, 2]                
  4                  -1  2     49664  ultralytics.nn.modules.block.C2f             [64, 64, 2, True]             
  5                  -1  1     73984  ultralytics.nn.modules.conv.Conv             [64, 128, 3, 2]               
  6                  -1  2    197632  ultralytics.nn.modules.block.C2f             [128, 128, 2, True]           
  7                  -1  1    295424  ultralytics

YOLOv8n-seg summary: 151 layers, 3,263,811 parameters, 3,263,795 gradients, 12.1 GFLOPs

Transferred 381/417 items from pretrained weights
Freezing layer 'model.22.dfl.conv.weight'
AMP: running Automatic Mixed Precision (AMP) checks...


...

train: Scanning /content/drive/MyDrive/computer_vision/segmentation_task_28_05_2025/dataset/train/labels.cache... 1100 images, 1100 backgrounds, 0 corrupt: 100%|██████████| 2200/2200 [00:00<?, ?it/s]


albumentations: Blur(p=0.01, blur_limit=(3, 7)), MedianBlur(p=0.01, blur_limit=(3, 7)), ToGray(p=0.01, method='weighted_average', num_output_channels=3), CLAHE(p=0.01, clip_limit=(1.0, 4.0), tile_grid_size=(8, 8))
val: Fast image access ✅ (ping: 0.5±0.2 ms, read: 0.0±0.0 MB/s, size: 3.0 KB)


val: Scanning /content/drive/MyDrive/computer_vision/segmentation_task_28_05_2025/dataset/val/labels.cache... 300 images, 300 backgrounds, 0 corrupt: 100%|██████████| 600/600 [00:00<?, ?it/s]


Plotting labels to /content/drive/MyDrive/computer_vision/segmentation_task_28_05_2025/runs/trial_0/labels.jpg... 
optimizer: Adam(lr=0.0008438170308626528, momentum=0.937) with parameter groups 66 weight(decay=0.0), 77 weight(decay=0.0029582801455634547), 76 bias(decay=0.0)
Image sizes 640 train, 640 val
Using 2 dataloader workers
Logging results to /content/drive/MyDrive/computer_vision/segmentation_task_28_05_2025/runs/trial_0
Starting training for 100 epochs...

      Epoch    GPU_mem   box_loss   seg_loss   cls_loss   dfl_loss  Instances       Size


      1/100      2.57G     0.9159      2.083      2.524      1.245         18        640:  22%|██▏       | 30/138 [03:46<11:02,  6.13s/it]